In [ ]:
!pip install transformers==3.5.0
!pip install nltk
!pip install biobert-pytorch
!pip install wget
!pip install cookiecutter

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD" -O biobert_weights && rm -rf /tmp/cookies.txt

!tar -xzf biobert_weights

!transformers-cli convert --model_type bert --tf_checkpoint biobert_v1.1_pubmed/model.ckpt-1000000 --config biobert_v1.1_pubmed/bert_config.json --pytorch_dump_output biobert_v1.1_pubmed/pytorch_model.bin

!mv biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json

--2020-12-14 17:02:18--  https://docs.google.com/uc?export=download&confirm=fZh3&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD
Resolving docs.google.com (docs.google.com)... 74.125.20.139, 74.125.20.113, 74.125.20.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.20.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-1o-docs.googleusercontent.com/docs/securesc/kdikajlk6prj1me7cr0jd12cfp1nsm24/kmm5m60qei8jjir3ch46j4iunnv772u6/1607965275000/13799006341648886493/13182991624626199490Z/1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD?e=download [following]
--2020-12-14 17:02:19--  https://doc-0o-1o-docs.googleusercontent.com/docs/securesc/kdikajlk6prj1me7cr0jd12cfp1nsm24/kmm5m60qei8jjir3ch46j4iunnv772u6/1607965275000/13799006341648886493/13182991624626199490Z/1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD?e=download
Resolving doc-0o-1o-docs.googleusercontent.com (doc-0o-1o-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting to 

In [ ]:
import os
import io
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForPreTraining
import nltk
from sklearn.model_selection import train_test_split
import re

from transformers import BertTokenizer, BertModel
from transformers import BertForTokenClassification, AdamW
from transformers import tokenization_utils_base
import torch
import torch.nn.functional as F
import torch.nn as nn
import pickle
from torch.utils.data import TensorDataset, DataLoader
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
TEXT_PATH="/content/drive/My Drive/MeasEval-main/data/train/dev/text"
TSV_PATH="/content/drive/My Drive/MeasEval-main/data/train/dev/tsv"
MEASURED_ENTITY_MODEL_PATH="/content/drive/My Drive/Model/biobert_measured_entity.pt"
MEASURED_QUANTITY_MODEL_PATH="/content/drive/My Drive/Model/quantity_mixedloss_t55_sh.pt"
MEASURED_PROPERTY_MODEL_PATH="/content/drive/My Drive/Model/biobert_measured_prop.pt"
QUALIFIER_PATH="/content/drive/My Drive/Model/qualifier_20f1_t995.pt"
COUNT_MEASUREMENT_MODEL_PATH="/content/drive/My Drive/Model/count_t95_f70_ep20_sirdata.pt"
INFORMATION_DETECTION_PATH="/content/drive/My Drive/Model/info_60f1_t325.pt"
UNIT_DETECTION_PATH="/content/drive/My Drive/Model/units_t75_96f1.pt"
VOCAB_PATH="/content/drive/My Drive/Model/vocab.Field"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_BioBert = BertModel.from_pretrained('/content/biobert_v1.1_pubmed')
tokenizer_BioBert = BertTokenizer(vocab_file='biobert_v1.1_pubmed/vocab.txt', do_lower_case=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
files=os.listdir(TEXT_PATH)

In [ ]:
test_text=[]
id=[]
j=1
for i in files:

  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  
  sent_text = nltk.sent_tokenize(text)
  for l in sent_text:
    test_text.append(l)
    id.append(j)
  j+=1

In [ ]:
test_text

['Principal component analysis of fungal TRFs for (A) all seven months combined and (B) for month 7 only.',
 'It is possible to explain such observations in general if we consider MEX to be above a region that has accelerated electrons upwards, in which case heavy-ions could also be accelerated downwards.',
 'This may explain the appearance of the very low DEF of heavy-ions at ∼400eV.',
 'However, due to the finite gyro-radius effect it is also plausible that the general behaviour of the heavy-ions flowing away from Mars may not change, even when passing an acceleration region.',
 'The gyroradius of heavy-ions with energies around ∼10eV at the location of the accelerated electron signature is around 100 km, which is of a similar spatial scale to the horizontal size of a closed crustal magnetic field line at 400 km.',
 'Therefore, it is possible the heavy-ions do not remain in the acceleration region long enough to experience its effects.',
 'Out of the total 689 events of electron prec

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      x = self.bert(sent_id, attention_mask=mask)
      embed = x[0]
      cls_hs = x[1]
      x,_ = self.bilstm(embed)
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.sigmoid(x)

      return x
      
bert_model = BERT_Arch(model_BioBert, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Measured Property

In [ ]:
l=0
NERdata=[]
for i in files:
  
  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv= os.path.join(TSV_PATH,i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text']]
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      for ent_doc in entity:
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          types.append(ent_doc[0])

      ent_sen.append(types)     
    
    
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer_BioBert)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))

    row.append(j)
    row.append(label)
    NERdata.append(row)
    
  
  
  
  

In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Id','Text','Entity'])

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [[]])[:512]
  print(textlabels[i])

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Quantity'], ['MeasuredEntity'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [

In [ ]:
inputs = tokenizer_BioBert(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
valseq = np.array(inputs['input_ids'])
valmask = np.array(inputs['attention_mask'])

In [ ]:
labels = {"MeasuredProperty": 0}
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),1));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      for k in range(len(textlabels[i][j])):
        if textlabels[i][j][k] =="MeasuredProperty":
          onehot_labels[i][j][labels[textlabels[i][j][k]]] = 1

In [ ]:
data = TensorDataset(torch.from_numpy(valseq), torch.from_numpy(valmask),torch.from_numpy(onehot_labels))

In [ ]:
batch_size = 38
loader = DataLoader(data, shuffle=True, batch_size = batch_size)

In [ ]:
bert_model = torch.load(MEASURED_PROPERTY_MODEL_PATH)
Measured_Property=[]

for s in test_text:
  tokenized_text = tokenizer_BioBert.tokenize(s)
  test_inputs = tokenizer_BioBert(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
  Measured_Property.append(temp[0])


In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          t1=0.3
          
          if np_out[i][j][k]>=t1:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------MEASURED PROPEPERTY RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.2577777777777778
--------MEASURED PROPEPERTY RESULTS--------
Accuracy:-0.9983609453517588
Modified Accuracy:-0.2577777777777778
Precision:-0.35802469135802467
Recall:-0.4793388429752066
F1 score:-0.40989399293286216


In [ ]:
Property=[]
for i in range(len(test_text)):
  tokenized_text = tokenizer_BioBert.tokenize(test_text[i])
  txt=[]
  label=[]
  # print(test_text[i])
  combine=""
  for j in range(len(tokenized_text)):
    if(Measured_Property[i][j+1] and Measured_Property[i][j] and j!=0):
      label.append(tokenized_text[j])
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
          combine=combine+tokenized_text[j][2:]
      
      else:
        if(len(combine)==0):
          combine=tokenized_text[j]
          continue
        
        if(tokenized_text[j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?","+","-","*","/","%")):
          combine=combine+tokenized_text[j]
          continue
        if(tokenized_text[j] in ("(",")") or combine in ("(",")")):
          continue
  
        start=[m.start() for m in re.finditer(combine,test_text[i])]
        end=[m.start() for m in re.finditer(tokenized_text[j],test_text[i])]
        flag=-1
        
        for s in start:
          for e in end:
            if(flag!=-1):
              break
            
            
            if(e-s-len(combine)+1==1):
              flag=1
            elif(e-s-len(combine)+1==2):
              flag=2
        
        if(flag==2):
          combine=combine+" "+tokenized_text[j]
        elif(flag==1):
          combine=combine+tokenized_text[j]
          
        else:
          txt.append(combine)
          combine=""

    elif(Measured_Property[i][j+1]):
      label.append(tokenized_text[j])
      
      if(len(combine)!=0):
        txt.append(combine)
      
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
        tokenized_text[j]=tokenized_text[j][2:]
      
      combine=tokenized_text[j]
    else:
      if(len(combine)!=0):
        txt.append(combine)
      
      combine=""
  if(len(combine)!=0):
        txt.append(combine)
  
  Property.append(txt)


Measured Entity

In [ ]:
l=0
NERdata=[]
for i in files:
  
  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv= os.path.join(TSV_PATH,i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text']]
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      for ent_doc in entity:
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          types.append(ent_doc[0])

      ent_sen.append(types)     
    
    
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer_BioBert)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))

    row.append(j)
    row.append(label)
    NERdata.append(row)
    
  
  
  
  

In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Id','Text','Entity'])

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [[]])[:512]
  print(textlabels[i])

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Quantity'], ['MeasuredEntity'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [

In [ ]:
inputs = tokenizer_BioBert(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
valseq = np.array(inputs['input_ids'])
valmask = np.array(inputs['attention_mask'])

In [ ]:
labels = {"MeasuredEntity": 0}
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),1));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      for k in range(len(textlabels[i][j])):
        if textlabels[i][j][k] =="MeasuredEntity":
          onehot_labels[i][j][labels[textlabels[i][j][k]]] = 1

In [ ]:
data = TensorDataset(torch.from_numpy(valseq), torch.from_numpy(valmask),torch.from_numpy(onehot_labels))

In [ ]:
batch_size = 38
loader = DataLoader(data, shuffle=True, batch_size = batch_size)

In [ ]:
bert_model = torch.load(MEASURED_ENTITY_MODEL_PATH)
Measured_Entity=[]
for s in test_text:
  tokenized_text = tokenizer_BioBert.tokenize(s)
  test_inputs = tokenizer_BioBert(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
  Measured_Entity.append(temp[0])

In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          t1=0.3
          
          if np_out[i][j][k]>=t1:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------MEASURED ENTITY RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.36796536796536794
--------MEASURED ENTITY RESULTS--------
Accuracy:-0.997134108040201
Modified Accuracy:-0.36796536796536794
Precision:-0.5414012738853503
Recall:-0.5345911949685535
F1 score:-0.5379746835443038


In [ ]:
Entity=[]
for i in range(len(test_text)):
  tokenized_text = tokenizer_BioBert.tokenize(test_text[i])
  txt=[]
  label=[]
  print(test_text[i])
  combine=""
  for j in range(len(tokenized_text)):
    if(Measured_Entity[i][j+1] and Measured_Entity[i][j] and j!=0):
      label.append(tokenized_text[j])
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
          combine=combine+tokenized_text[j][2:]
      
      else:
        if(len(combine)==0):
          combine=tokenized_text[j]
          continue
        
        if(tokenized_text[j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?","+","-","*","/","%")):
          combine=combine+tokenized_text[j]
          continue
        if(tokenized_text[j] in ("(",")") or combine in ("(",")")):
          continue
  
        start=[m.start() for m in re.finditer(combine,test_text[i])]
        end=[m.start() for m in re.finditer(tokenized_text[j],test_text[i])]
        flag=-1
        
        for s in start:
          for e in end:
            if(flag!=-1):
              break
            
            
            if(e-s-len(combine)+1==1):
              flag=1
            elif(e-s-len(combine)+1==2):
              flag=2
        
        if(flag==2):
          combine=combine+" "+tokenized_text[j]
        elif(flag==1):
          combine=combine+tokenized_text[j]
          
        else:
          txt.append(combine)
          combine=""

    elif(Measured_Entity[i][j+1]):
      label.append(tokenized_text[j])
      
      if(len(combine)!=0):
        txt.append(combine)
      
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
        tokenized_text[j]=tokenized_text[j][2:]
      
      combine=tokenized_text[j]
    else:
      if(len(combine)!=0):
        txt.append(combine)
      
      combine=""
  if(len(combine)!=0):
        txt.append(combine)
  
  
  Entity.append(txt)
        


Principal component analysis of fungal TRFs for (A) all seven months combined and (B) for month 7 only.
It is possible to explain such observations in general if we consider MEX to be above a region that has accelerated electrons upwards, in which case heavy-ions could also be accelerated downwards.
This may explain the appearance of the very low DEF of heavy-ions at ∼400eV.
However, due to the finite gyro-radius effect it is also plausible that the general behaviour of the heavy-ions flowing away from Mars may not change, even when passing an acceleration region.
The gyroradius of heavy-ions with energies around ∼10eV at the location of the accelerated electron signature is around 100 km, which is of a similar spatial scale to the horizontal size of a closed crustal magnetic field line at 400 km.
Therefore, it is possible the heavy-ions do not remain in the acceleration region long enough to experience its effects.
Out of the total 689 events of electron precipitation signatures, 85 w

Quantity

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
l=0
NERdata=[]
for i in files:
  
  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv= os.path.join(TSV_PATH,i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text']]
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      for ent_doc in entity:
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          types.append(ent_doc[0])

      ent_sen.append(types)     
    
    
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))

    row.append(j)
    row.append(label)
    NERdata.append(row)
    
  
  
  
  

In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Id','Text','Entity'])

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [[]])[:512]
  print(textlabels[i])

[[], [], [], [], [], [], [], [], [], [], [], [], [], ['Quantity'], ['MeasuredEntity'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [

In [ ]:
inputs = tokenizer(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
valseq = np.array(inputs['input_ids'])
valmask = np.array(inputs['attention_mask'])

In [ ]:
labels = {"Quantity": 0}
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),1));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      for k in range(len(textlabels[i][j])):
        if textlabels[i][j][k] =="Quantity":
          onehot_labels[i][j][labels[textlabels[i][j][k]]] = 1

In [ ]:
data = TensorDataset(torch.from_numpy(valseq), torch.from_numpy(valmask),torch.from_numpy(onehot_labels))

In [ ]:
batch_size = 38
loader = DataLoader(data, shuffle=True, batch_size = batch_size)

In [ ]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)
bert_model = torch.load(MEASURED_QUANTITY_MODEL_PATH)
Measured_Quantity=[]
for s in test_text:
  tokenized_text = tokenizer.tokenize(s)
  test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
  Measured_Quantity.append(temp[0])
  

In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          t1=0.55
          
          if np_out[i][j][k]>=t1:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------QUANTITY RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.8151041666666666
--------QUANTITY RESULTS--------
Accuracy:-0.9993031564070352
Modified Accuracy:-0.8151041666666666
Precision:-0.9072463768115943
Recall:-0.8892045454545454
F1 score:-0.8981348637015784


In [ ]:
Quantity=[]
for i in range(len(test_text)):
  tokenized_text = tokenizer.tokenize(test_text[i])
  txt=[]
  label=[]
  
  combine=""
  for j in range(len(tokenized_text)):
    if(Measured_Quantity[i][j+1] and Measured_Quantity[i][j] and j!=0):
      label.append(tokenized_text[j])
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
          combine=combine+tokenized_text[j][2:]
      
      else:
        if(len(combine)==0):
          combine=tokenized_text[j]
          continue
        
        if(tokenized_text[j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?","+","-","*","/","%")):
          combine=combine+tokenized_text[j]
          continue
        if(tokenized_text[j] in ("(",")") or combine in ("(",")")):
          print(combine)
          continue
        start=[m.start() for m in re.finditer(combine,test_text[i])]
        end=[m.start() for m in re.finditer(tokenized_text[j],test_text[i])]
        flag=-1
        
        for s in start:
          for e in end:
            if(flag!=-1):
              break
            
            
            if(e-s-len(combine)+1==1):
              flag=1
            elif(e-s-len(combine)+1==2):
              flag=2
        
        if(flag==2):
          combine=combine+" "+tokenized_text[j]
        elif(flag==1):
          combine=combine+tokenized_text[j]
          
        else:
          txt.append(combine)
          combine=""

    elif(Measured_Quantity[i][j+1]):
      label.append(tokenized_text[j])
      
      if(len(combine)!=0):
        txt.append(combine)
      
      if(len(tokenized_text[j])>=2 and tokenized_text[j][:2]=="##"):
        tokenized_text[j]=tokenized_text[j][2:]
      
      combine=tokenized_text[j]
    else:
      if(len(combine)!=0):
        txt.append(combine)
      
      combine=""
        
  if(len(combine)!=0):
        txt.append(combine)
  
               
  
  Quantity.append(txt)
  
  

u mol−1
-bit


In [ ]:
Quantity

[['seven'],
 [],
 ['[UNK]'],
 [],
 ['around', 'around 100 km,', '400 km.'],
 [],
 ['689', '85'],
 ['12%', '[UNK]%'],
 ['37'],
 ['just 5%', '[UNK]%'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['20', ',', '111', 'a'],
 ['63', 'a', '20 wt%'],
 ['[UNK]', 'u mol−1', '5 and 300', '.'],
 [],
 ['two'],
 ['last 200 years'],
 [],
 [],
 ['10%'],
 ['298', '.'],
 [],
 ['[UNK]', '25:00', '01:25:40', ','],
 [],
 [],
 [],
 [],
 [],
 ['192 s'],
 ['15 or less'],
 [],
 [],
 ['05:28', '05:47'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['from 7 to 20%', '10 year', 'over 48 years', '13%'],
 ['approximately', '%'],
 [],
 [],
 ['001'],
 [],
 ['< 9 m'],
 ['greater than 10%', '5%'],
 [],
 [],
 ['7-month'],
 [],
 ['7 month'],
 [],
 [],
 ['9 or 4.8 months'],
 [],
 [],
 [],
 ['12 min,', '2-min'],
 ['approximately 60', '.'],
 [],
 [],
 [],
 [],
 [],
 ['about −100', '.'],
 ['100 mg/', '1 mg/', '10 mg/', '0.1 mg/', '.'],
 [],
 ['1, 5, 10, 20 and 50 μg/', '1%'],
 ['%'],
 [],
 [],
 ['10 wt%', '20 wt%'],
 [],
 [],
 [],
 [],
 ['0.296 μm.'],

Qualifier

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
l=0
NERdata=[]
for i in files:
  
  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv= os.path.join(TSV_PATH,i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text']]
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      for ent_doc in entity:
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          types.append(ent_doc[0])

      ent_sen.append(types)     
    
    
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))

    row.append(j)
    row.append(label)
    NERdata.append(row)
    
  
  
  
  

In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Id','Text','Entity'])

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [[]])[:512]

In [ ]:
inputs = tokenizer(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
valseq = np.array(inputs['input_ids'])
valmask = np.array(inputs['attention_mask'])

In [ ]:
labels = {"Qualifier": 0}
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),1));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      for k in range(len(textlabels[i][j])):
        if textlabels[i][j][k] =="Qualifier":
          onehot_labels[i][j][labels[textlabels[i][j][k]]] = 1

In [ ]:
data = TensorDataset(torch.from_numpy(valseq), torch.from_numpy(valmask),torch.from_numpy(onehot_labels))

In [ ]:
batch_size = 38
loader = DataLoader(data, shuffle=True, batch_size = batch_size)

In [ ]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)
bert_model = torch.load(QUALIFIER_PATH)

In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          t1=0.995
          
          if np_out[i][j][k]>=t1:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------QUALIFIER RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.11242603550295859
--------QUALIFIER RESULTS--------
Accuracy:-0.9985277952261307
Modified Accuracy:-0.11242603550295859
Precision:-0.17272727272727273
Recall:-0.24358974358974358
F1 score:-0.20212765957446807


In [ ]:
# bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
# bert_model = bert_model.to(device)
# bert_model = torch.load(QUALIFIER_PATH)
# Qualifier=[]
# for s in test_text:
#   tokenized_text = tokenizer.tokenize(s)
#   test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
#   bert_model.eval()
#   temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
#   Qualifier.append(temp[0])
  

Count vs Measurement

In [ ]:
l=0
NERdata=[]
for i in files:
  
  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv=os.path.join(TSV_PATH,i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text','other']]
  entity=entity[entity['annotType']=='Quantity']
  
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      # other=[]
      for ent_doc in entity:
        
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          dictio={}
          if(not pd.isnull(ent_doc[4])):
            dictio=ast.literal_eval(ent_doc[4])

          
          if('mods' in dictio):
            info=dictio['mods']
            
            if('IsCount' in info):
              types.append("Count")
              
            else:
              types.append("Measurement")
          else:
            types.append("Measurement")


        # print(word,types,dictio)
      
      ent_sen.append(types)     
    
    # print(ent_sen)
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))
    # print(j)
    # print(label)
    row.append(j)
    row.append(label)
    
    NERdata.append(row)
    
  
  
  
  

In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Id','Text','Entity'])

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [np.zeros(3)])[:512]

In [ ]:
inputs = tokenizer(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
train_seq = np.array(inputs['input_ids'])
train_mask = np.array(inputs['attention_mask'])

In [ ]:
labels = {"Count":1, "Measurement": 2}

In [ ]:
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),1));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      if (len(textlabels[i][j])==0):
        onehot_labels[i][j][0]=0
      elif (textlabels[i][j][0]=="Count"):
        onehot_labels[i][j][0] = 1

In [ ]:
onehot_labels2 = np.zeros((len(textlabels), len(textlabels[0]),1));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      if (len(textlabels[i][j])==0):
        onehot_labels2[i][j][0]=0
      elif (textlabels[i][j][0]=="Measurement"):
        onehot_labels2[i][j][0] = 1

In [ ]:
unit_data = TensorDataset(torch.from_numpy(train_seq), torch.from_numpy(train_mask),torch.from_numpy(onehot_labels))
meas_data = TensorDataset(torch.from_numpy(train_seq), torch.from_numpy(train_mask),torch.from_numpy(onehot_labels2))

In [ ]:
batch_size = 38
unit_loader = DataLoader(unit_data, shuffle=True, batch_size = batch_size)
meas_loader = DataLoader(meas_data, shuffle=True, batch_size = batch_size)

In [ ]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)
bert_model = torch.load(MEASURED_QUANTITY_MODEL_PATH)

In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in meas_loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          t1=0.55
          
          if np_out[i][j][k]>=t1:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1




In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.bilstm = nn.LSTM(embed_dim, hidden_dim, n_layers,dropout=drop_prob,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,256)
      self.r1=nn.Tanh()
      self.fc2 = nn.Linear(256,out_dim)
      self.sf = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      embed, cls_hs = self.bert(sent_id, attention_mask=mask)
      x,_ = self.bilstm(embed)
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.r1(x)
      x = self.fc2(x)
      x = self.sf(x)

      return x

bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
bert_model = torch.load(COUNT_MEASUREMENT_MODEL_PATH)
count_measurement=[]
for s in test_text:
  tokenized_text = tokenizer.tokenize(s)
  test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.95
  count_measurement.append(temp[0])

In [ ]:

for seq, mask, y in unit_loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if(np_out[i,j,0]>0.95):
          np_out[i,j,0] = 1
        else:
          np_out[i,j,0] = 0
        

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n-1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg - 1
          if np_out[i,j,k]>0:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den - 1
          if np_act[i,j,k]>0:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den - 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("--------COUNT/MEASUREMENT RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

--------COUNT/MEASUREMENT RESULTS--------
Accuracy:-0.9996515542642888
Modified Accuracy:-0.814621409921671
Precision:-0.8939828080229226
Recall:-0.9017341040462428
F1 score:-0.897841726618705


In [ ]:
counts = []
for i in range(len(test_text)):
  tokenized_text = tokenizer.tokenize(test_text[i])
  txt=[]
  for j in range(len(tokenized_text)):
    if(count_measurement[i][j+1]):
      txt.append(tokenized_text[j])   
  counts.append(txt)

In [ ]:
measurement = []
for i in range(len(counts)):
  measurement.append(list(set(Quantity[i]).difference(set(counts[i]))))

In [ ]:
measurement

[[],
 [],
 ['[UNK]'],
 [],
 ['around 100 km,', 'around', '400 km.'],
 [],
 ['689'],
 ['12%', '[UNK]%'],
 [],
 ['just 5%', '[UNK]%'],
 [],
 [],
 [],
 [],
 [],
 [],
 [',', '111', 'a', '20'],
 ['63', '20 wt%', 'a'],
 ['u mol−1', '[UNK]', '5 and 300', '.'],
 [],
 [],
 ['last 200 years'],
 [],
 [],
 ['10%'],
 ['298', '.'],
 [],
 ['[UNK]', '01:25:40', ',', '25:00'],
 [],
 [],
 [],
 [],
 [],
 ['192 s'],
 ['15 or less'],
 [],
 [],
 ['05:47', '05:28'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['13%', '10 year', 'from 7 to 20%', 'over 48 years'],
 ['%', 'approximately'],
 [],
 [],
 ['001'],
 [],
 ['< 9 m'],
 ['greater than 10%', '5%'],
 [],
 [],
 ['7-month'],
 [],
 ['7 month'],
 [],
 [],
 ['9 or 4.8 months'],
 [],
 [],
 [],
 ['12 min,', '2-min'],
 ['approximately 60', '.'],
 [],
 [],
 [],
 [],
 [],
 ['.', 'about −100'],
 ['1 mg/', '100 mg/', '0.1 mg/', '10 mg/', '.'],
 [],
 ['1%', '1, 5, 10, 20 and 50 μg/'],
 ['%'],
 [],
 [],
 ['10 wt%', '20 wt%'],
 [],
 [],
 [],
 [],
 ['0.296 μm.'],
 ['0.18 μm.'],
 [],
 

Extrainformation

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.n_layers = n_layers
      self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      
      # embed,cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.bert(sent_id, attention_mask=mask)
      embed=x[0]
      cls_hs=x[1]
      
      x,_ = self.bilstm(embed)
      
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.sigmoid(x)

      return x

bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)



In [ ]:
files=os.listdir(TEXT_PATH)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
l=0
NERdata=[]
for i in files:
  
  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text=f.read()
  pathtsv=os.path.join(TSV_PATH,i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  entity=filetsv[['annotType','startOffset','endOffset','text','other']]
  entity=entity[entity['annotType']=='Quantity']
  
  entity = entity.to_numpy().tolist()

  ent=0
  offset=0
  sent_text = nltk.sent_tokenize(text)
  
  for j in sent_text:
    row=[]
    # row.append(i[:-4])
    
    
    ent_sen=[]
    ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
    start=[]
    
    for word in ret:
      types=[]
      # other=[]
      for ent_doc in entity:
        
        if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
          dictio={}
          if(not pd.isnull(ent_doc[4])):
            dictio=ast.literal_eval(ent_doc[4])

          
          if('mods' in dictio):
            info=dictio['mods']
            for i in info:
              types.append(i)
            
              
            
            

        
        
      ent_sen.append(types)     
    
    # print(ent_sen)
    sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
    label.insert(0,[])
    label.append([])
    offset+=(1+len(j))
    row.append(j)
    row.append(label)
    
    NERdata.append(row)

In [ ]:
NERdf=pd.DataFrame (NERdata, columns = ['Text','Entity'])

In [ ]:
text=NERdf['Text'].tolist()
textlabels=NERdf['Entity'].tolist()

for i in range(len(textlabels)):
  textlabels[i]=(textlabels[i] + 512 * [[]])[:512]


In [ ]:
inputs = tokenizer(text,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")

In [ ]:
train_seq = np.array(inputs['input_ids'])
train_mask = np.array(inputs['attention_mask'])

In [ ]:
labels = {'HasTolerance': 0,
 'IsApproximate': 1,
 'IsCount': 2,
 'IsList': 3,
 'IsMean': 4,
 'IsMeanHasSD': 5,
 'IsMeanHasTolerance': 6,
 'IsMeanIsRange': 7,
 'IsMedian': 8,
 'IsRange': 9,
 'IsRangeHasTolerance': 10}

In [ ]:
onehot_labels = np.zeros((len(textlabels), len(textlabels[0]),11));
for i in range(len(textlabels)):
  for j in range(len(textlabels[0])):
      for k in range(len(textlabels[i][j])):
        onehot_labels[i][j][labels[textlabels[i][j][k]]] = 1

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
data = TensorDataset(torch.from_numpy(train_seq), torch.from_numpy(train_mask),torch.from_numpy(onehot_labels))


In [ ]:
batch_size = 32
loader = DataLoader(data, shuffle=True, batch_size = batch_size)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
bert_model = torch.load(INFORMATION_DETECTION_PATH)
info=[]
for s in test_text:
  tokenized_text = tokenizer.tokenize(s)
  test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
  bert_model.eval()
  temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.3
  info.append(temp[0])


In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.to(device), mask.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    #print(np_out)
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(11):
          if np_out[i][j][k]>=0.325:
            np_out[i][j][k]=1
          else:
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(11):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------ADDITIONAL INFO RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.5490196078431373
--------ADDITIONAL INFO RESULTS--------
Accuracy:-0.9998786546368205
Modified Accuracy:-0.4309623430962343
Precision:-0.5659340659340659
Recall:-0.64375
F1 score:-0.6023391812865497


In [ ]:
idx2labels = {0:'HasTolerance',
 1:'IsApproximate',
 2:'IsCount',
 3:'IsList',
 4:'IsMean',
 5:'IsMeanHasSD',
 6:'IsMeanHasTolerance',
 7:'IsMeanIsRange',
 8:'IsMedian',
 9:'IsRange',
 10:'IsRangeHasTolerance'}

In [ ]:
for i in range(len(test_text)):
  tokenized_text = tokenizer.tokenize(test_text[i])
  txt=[]
  print(test_text[i])
  
  for j in range(len(tokenized_text)):
    for k in range(11):
      if(info[i][j+1][k]):
        txt.append(tokenized_text[j]+" "+idx2labels[k])   
  print(txt)

Principal component analysis of fungal TRFs for (A) all seven months combined and (B) for month 7 only.
['seven IsCount']
It is possible to explain such observations in general if we consider MEX to be above a region that has accelerated electrons upwards, in which case heavy-ions could also be accelerated downwards.
[]
This may explain the appearance of the very low DEF of heavy-ions at ∼400eV.
[]
However, due to the finite gyro-radius effect it is also plausible that the general behaviour of the heavy-ions flowing away from Mars may not change, even when passing an acceleration region.
[]
The gyroradius of heavy-ions with energies around ∼10eV at the location of the accelerated electron signature is around 100 km, which is of a similar spatial scale to the horizontal size of a closed crustal magnetic field line at 400 km.
['around IsApproximate', '[UNK] IsApproximate', 'around IsApproximate', '100 IsApproximate', 'km IsApproximate', ', IsApproximate', 'which IsApproximate']
Therefore

Unit Detection

In [ ]:
file = open(VOCAB_PATH,'rb')
TEXT = pickle.load(file)
vocab_size=len(TEXT.vocab)
class bilstm(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_size = output_size
        
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                         batch_first=True, bidirectional=True)
        
        self.dropout = nn.Dropout(0)
        self.fc = nn.Linear(hidden_dim*2,output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        embeds = self.embedding(x)
        lstm, (hn,cn)= self.lstm(embeds)
        
        output = self.dropout(lstm)
        output = self.fc(output)
        output = self.sigmoid(output) 
        return output
device = torch.device("cpu")
model = bilstm(vocab_size, 1, 100, 32, 1)
model = model.to(device)

In [ ]:
model = torch.load(UNIT_DETECTION_PATH)

In [ ]:
train=[]
for i in files:
  
  pathtsv=os.path.join(TSV_PATH,i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  
  entity=filetsv[['annotType','startOffset','endOffset','text','other']]
  quant=entity[entity['annotType']=='Quantity']
  quant=quant[['text','other']]
  quant = quant.to_numpy().tolist()
  train.append(quant)
  
  
  
  

In [ ]:
X=[]
label=[]
for i in train:
  for j in i:
    temp=[]
  
    X.append(j[0])
    
    
    
    
    other=j[1]
    
    if(not pd.isnull(other)):
      dictio=ast.literal_eval(other) 
    
      if 'unit' in dictio:
        t=j[0].find(dictio['unit'])
        
        t=[m.start() for m in re.finditer(dictio['unit'],j[0])]
        sz=len(dictio['unit'])
        span=[]
        for k in t:
          span.append((k,k+sz))
        
        
        label.append(span)
        

      else:
        label.append([])
    else:
      label.append([])

    

In [ ]:
X_test=[]
y_test=[]
for i in range(len(X)):

  character=list(X[i]);
  X_test.append(character)
  l=np.zeros(64)
  
  for k in label[i]:
    l[k[0]:k[1]]=1;
  y_test.append(l)

In [ ]:
X_test=TEXT.process(X_test)[0]
row=X_test.shape[1]
X_test=F.pad(input=X_test, pad=(0, 64-row, 0, 0), mode='constant', value=1)
test_data = TensorDataset(X_test,torch.from_numpy(np.array(y_test)))

In [ ]:
batch_size = 38
test_loader = DataLoader(test_data, shuffle=True, batch_size = batch_size)

In [ ]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, y in test_loader:
    model.zero_grad()
    model.eval()
    y_pred = model(seq.cpu())
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    
    
    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
          t=0.75
          if np_out[i][j]>=t:
            np_out[i][j]=1
          else:
            np_out[i][j]=0


    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
          if np_out[i,j] == np_act[i,j]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j])==1 or (np_act[i,j])==1:
            if (np_out[i,j])==1 and (np_act[i,j])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j]==1:
            if np_out[i,j]==np_act[i,j]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j]==1:
            if np_out[i,j]==np_act[i,j]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)


print("--------UNIT DETECTION RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

--------UNIT DETECTION RESULTS--------
Accuracy:-0.9971301020408163
Modified Accuracy:-0.918552036199095
Precision:-0.9712918660287081
Recall:-0.9441860465116279
F1 score:-0.9575471698113208


In [ ]:
UNITS=[]

for i in Quantity:
  unit=[]
  for j in i:
    X_train=[]
    X_train.append(text_field.process(j)[0].numpy())
    
    X_train=np.array(X_train)

    
    X_train=torch.from_numpy(X_train)
    X_train=X_train[:,:,0]

    
    row=X_train.shape[1]
    
    
    X_train=F.pad(input=X_train, pad=(0, 64-row, 0, 0), mode='constant', value=1)
    
    model.zero_grad()
    model.eval()
    y_pred = model(X_train.to(device))
    
    np_out = y_pred.cpu().data.numpy()
    
    np_out=np_out>=0.75
    un=""
    for k in range(len(np_out[0])):
      if(np_out[0][k]):
        un=un+j[k]
    unit.append(un)

  UNITS.append(unit)
    
    
    

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
Quantity

[['seven'],
 [],
 ['[UNK]'],
 [],
 ['around', 'around 100 km,', '400 km.'],
 [],
 ['689', '85'],
 ['12%', '[UNK]%'],
 ['37'],
 ['just 5%', '[UNK]%'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['20', ',', '111', 'a'],
 ['63', 'a', '20 wt%'],
 ['[UNK]', 'u mol−1', '5 and 300', '.'],
 [],
 ['two'],
 ['last 200 years'],
 [],
 [],
 ['10%'],
 ['298', '.'],
 [],
 ['[UNK]', '25:00', '01:25:40', ','],
 [],
 [],
 [],
 [],
 [],
 ['192 s'],
 ['15 or less'],
 [],
 [],
 ['05:28', '05:47'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['from 7 to 20%', '10 year', 'over 48 years', '13%'],
 ['approximately', '%'],
 [],
 [],
 ['001'],
 [],
 ['< 9 m'],
 ['greater than 10%', '5%'],
 [],
 [],
 ['7-month'],
 [],
 ['7 month'],
 [],
 [],
 ['9 or 4.8 months'],
 [],
 [],
 [],
 ['12 min,', '2-min'],
 ['approximately 60', '.'],
 [],
 [],
 [],
 [],
 [],
 ['about −100', '.'],
 ['100 mg/', '1 mg/', '10 mg/', '0.1 mg/', '.'],
 [],
 ['1, 5, 10, 20 and 50 μg/', '1%'],
 ['%'],
 [],
 [],
 ['10 wt%', '20 wt%'],
 [],
 [],
 [],
 [],
 ['0.296 μm.'],

In [ ]:
Property

[[],
 [],
 ['DEF'],
 [],
 ['gyroradius of heavy-ions',
  'energies',
  'location',
  'scale',
  'horizontal size',
  'line'],
 [],
 [],
 ['signatures occurring'],
 [],
 ['the peripheral acceleration of heavy-ions less common'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['value'],
 ['the compressive true yield stress'],
 ['temperature independent paramagnetic', 'ties'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['acceleration of heavy-ions'],
 [],
 [],
 [],
 [],
 [],
 ['observed'],
 ['mass channel number', 'maximum ion count'],
 [],
 ['data set'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['reported prevalence', 'year incidence rate in', 'aged over 48 years'],
 [],
 [],
 [],
 ['th'],
 [],
 ['long'],
 ['clay mineral', 'uncertainty', 'level'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['cubate'],
 [],
 [],
 [],
 ['time interval'],
 ['gradient', 'energy'],
 [],
 [],
 [],
 [],
 [],
 ['toughened', 'pox'],
 ['concentrations', 'concentrations'],
 [],
 ['final standard concentrations'],
 ['ni', 'hydrogen'],
 [],
 [],


In [ ]:
Entity

[['component analysis of fungal TRF', 'months'],
 [],
 ['heavy'],
 [],
 ['gyroradius of heavy-ions',
  'electron',
  'closed crustal magnetic field line'],
 [],
 ['events'],
 ['precipitation signatures', 'MEX orbits.'],
 ['signatures'],
 ['acceleration', 'heavy-', 'precipitation signatures', 'MEX orbits'],
 [],
 ['pressive true yield'],
 [],
 [],
 ['-CSR particle'],
 [],
 ['y'],
 ['compressive true yield', 'S-CSR-', 'y polymer'],
 ['samples 1', '2', 'para'],
 [],
 ['marine cores'],
 [],
 [],
 [],
 ['largest group', 'MEX orbits.'],
 [],
 [],
 ['heavy-ions'],
 [],
 [],
 ['ions'],
 [],
 [],
 ['ions', 'cycle'],
 ['mass channel number', 'maximum ion count'],
 [],
 ['set'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['ni', 'general population,', 'incidence rate in adults'],
 ['population', 'tinnitus'],
 [],
 [],
 ['001 basal illite peak'],
 [],
 ['cores'],
 ['clay mineral values', 'uncertainty'],
 [],
 [],
 [],
 ['soils'],
 [],
 [],
 [],
 [],
 [],
 ['panel'],
 [],
 ['ELS spectrogram', '“inverted-V”

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
'''
lis_en = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
for i in range(len(test_text)):
  tag={}
  embed={}

    
  for j in range(len(Quantity[i])):
    L=test_text[i].lower()
    if L.find(Quantity[i][j]):
      start_q = L.index(Quantity[i][j])
      end_q = len(Quantity[i][j]) + start_q - 1
      tag["Q-"+str(j+1)]=[start_q,end_q,Quantity[i][j]]
      embed["Q-"+str(j+1)]=np.zeros(768)
  
  for j in range(len(Property[i])):
    test_text[i]=test_text[i].lower()
    if test_text[i].find(Property[i][j]):
      
      if(Property[i][j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?",")","+","-","*","/","%")):
        continue
      start=[m.start() for m in re.finditer(Property[i][j],test_text[i])]
      
      if(len(start)>0):
        start_q = start[0]
        end_q = len(Property[i][j]) + start_q - 1
        tag["P-"+str(j+1)]=[start_q,end_q,Property[i][j]]
        embed["P-"+str(j+1)]=np.zeros(768)

  for j in range(len(Entity[i])):
    
    if test_text[i].find(Entity[i][j]):
      if(Entity[i][j] in ("!", "," ,"\'" ,";" ,'(',"\"", ".", "-" ,"?",")","+","-","*","/","%")):
        continue
      print(Entity[i][j],test_text[i])
      start=[m.start() for m in re.finditer(Entity[i][j],test_text[i])]
      # print(start,test_text[i],Entity[i][j])
      if(len(start)>0):
        start_q = start[0]
        end_q = len(Entity[i][j]) + start_q - 1
        tag["E-"+str(j+1)]=[start_q,end_q,Entity[i][j]]
        embed["E-"+str(j+1)]=np.zeros(768)

  
  
  j=test_text[i]
  ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
  entities=[]
  for word in ret:
    temp=[]
    for key,ent_doc in tag.items():
      if(word[1]>=ent_doc[0] and word[2]<=ent_doc[1]):
        temp.append(key)
    entities.append(temp)
  

  sen,label=tokenize_and_preserve_labels(j.split(' '),entities,tokenizer)
  label.insert(0,[])
  label.append([])
  inputs = tokenizer(j, return_tensors="pt")
  outputs = model(**inputs)
  
  last_hidden_states = outputs[0]

  for i in range(len(label)):
    if(len(label[i])==0):
      continue
    else:
      for k in range(len(label[i])):
        embed[label[i][k]]=embed[label[i][k]]+last_hidden_states[0][i].detach().numpy()
  
  data=[]

  for key,val in tag.items():
    if(key[0]=='Q'):
      data.append(["Quantity",embed[key],val[2]])
    elif(key[0]=='P'):
      data.append(["MeasuredProperty",embed[key],val[2]])
    else:
      data.append(["MeasuredEntity",embed[key],val[2]])    
  lis_en.append(data)
'''

'\nlis_en = []\ntokenizer = BertTokenizer.from_pretrained(\'bert-base-uncased\')\nmodel = BertModel.from_pretrained(\'bert-base-uncased\')\nfor i in range(len(test_text)):\n  tag={}\n  embed={}\n\n    \n  for j in range(len(Quantity[i])):\n    L=test_text[i].lower()\n    if L.find(Quantity[i][j]):\n      start_q = L.index(Quantity[i][j])\n      end_q = len(Quantity[i][j]) + start_q - 1\n      tag["Q-"+str(j+1)]=[start_q,end_q,Quantity[i][j]]\n      embed["Q-"+str(j+1)]=np.zeros(768)\n  \n  for j in range(len(Property[i])):\n    test_text[i]=test_text[i].lower()\n    if test_text[i].find(Property[i][j]):\n      \n      if(Property[i][j] in ("!", "," ,"\'" ,";" ,""", ".", "-" ,"?",")","+","-","*","/","%")):\n        continue\n      start=[m.start() for m in re.finditer(Property[i][j],test_text[i])]\n      \n      if(len(start)>0):\n        start_q = start[0]\n        end_q = len(Property[i][j]) + start_q - 1\n        tag["P-"+str(j+1)]=[start_q,end_q,Property[i][j]]\n        embed["P-"+s

In [ ]:
#lis_en[4]

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
inputA = layers.Input(shape=(768,))
inputB = layers.Input(shape=(768,))
inputC = layers.Input(shape=(1,))
inputD = layers.Input(shape = (1,))

x = layers.Dense(128, activation="relu")(inputA)
x = layers.Dense(64, activation="relu")(x)

y = layers.Dense(128, activation="relu")(inputB)
y = layers.Dense(64, activation="relu")(y)

c = layers.Dense(1, activation="relu")(inputC)

d = layers.Dense(1, activation="relu")(inputD)


com = layers.concatenate([x, y, c, d])
z = layers.Dense(32, activation="relu")(com)
z = layers.Dense(4,  activation='softmax')(z)
model = keras.Model(inputs=[inputA, inputB, inputC, inputD], outputs=z)

In [ ]:
weights = [15.593220338983052, 10.415094339622641, 33.65853658536585, 2.323232323232323]

In [ ]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):

    weights = K.variable(weights)
        
    def loss(y_true, y_pred):

        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [ ]:
model.compile(optimizer='adam',
              loss= weighted_categorical_crossentropy(weights),
              metrics=['accuracy'])

In [ ]:
from keras.models import load_model
model.load_weights('/content/drive/My Drive/Model/rel_wt_new.h5')

In [ ]:
tok2idx = {}
tok2idx["Quantity"] = 1
tok2idx["MeasuredEntity"] = 2
tok2idx["MeasuredProperty"] = 3
tok2idx["Qualifier"]=4

In [ ]:
cd /content/drive/My Drive/MeasEval-main/data/train

In [ ]:
def data_read(path_to_file,text_path,model,tokenizer):
  y_train=[]
  l=0
  for i in path_to_file:
   #for j in :
     #i=path_to_file[j] 
    pathtxt=os.path.join(text_path,'text',i[:-3]+'txt')
    
    if (not os.path.isfile(pathtxt)):
      continue 
    
    with open(pathtxt, 'r') as f:
      text=f.read()
    pathtsv=os.path.join(text_path+'/tsv',i[:-3]+'tsv')
    if(not(os.path.exists(pathtsv))):
      continue
    filetsv=pd.read_csv(pathtsv,sep = '\t')
    data=filetsv[['annotType','startOffset','endOffset','text','annotId']]
    entity = data.to_numpy().tolist()
    tags={}
    ent=0
    offset=0
    sent_text = nltk.sent_tokenize(text)
    para=[]
    for j in sent_text:
      row=[]
      
      
      
      ent_sen=[]
      ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
      start=[]
      
      for word in ret:
        types=[]
        for ent_doc in entity:
          if(offset+word[1]>=ent_doc[1] and offset+word[2]<=ent_doc[2]):
            types.append(ent_doc[4])
            tags[ent_doc[4]]=np.zeros(768)
        ent_sen.append(types)     
      
      
      sen,label=tokenize_and_preserve_labels(j.split(' '),ent_sen,tokenizer)
      label.insert(0,[])
      label.append([])

      inputs = tokenizer(j, return_tensors="pt")
      outputs = model(**inputs)
      
      
      
      last_hidden_states = outputs.last_hidden_state
      
      for lbl in range(len(label)):
        for lbl2 in range(len(label[lbl])):
          if(label[lbl][lbl2] in tags):
            tags[label[lbl][lbl2]]+=last_hidden_states[0][lbl].detach().numpy()
          else:
            tags[label[lbl][lbl2]]=last_hidden_states[0][lbl].detach().numpy()
            
            

      offset+=(1+len(j))
      
      
    entity=filetsv[['annotType','annotId','startOffset','endOffset','text','other']]
    quant=entity[entity['annotType']!='Quantity']
    sent_text = nltk.sent_tokenize(text)
    quant=quant[['annotType','annotId','startOffset','endOffset','text','other']].to_numpy().tolist()
    paragraph=[]
    for qu in quant:
      other=qu[5]
      #print(qu)
      
      qu=qu[:-1]
      relation=[]
      if(not pd.isnull(other)):
        dictio=ast.literal_eval(other) 
        if 'HasQuantity' in dictio:
          id=dictio['HasQuantity']
          relation.append(qu[1])
          relation.append(id)
          relation.append(tok2idx[qu[0]])
          relation.append('HasQuantity')
          relation.append(1)
          
          
        if 'HasProperty' in dictio:
          id=dictio['HasProperty']
          relation.append(qu[1])
          relation.append(id)
          relation.append(tok2idx[qu[0]])
          relation.append('HasProperty')
          relation.append(3)
          
          
        if 'Qualifies' in dictio:
          id=dictio['Qualifies']          
          relation.append(qu[1])
          relation.append(id)
          relation.append(tok2idx[qu[0]])
          relation.append('Qualifies')
          relation.append(1)
          
        
        paragraph.append(relation)
    
    key=list(tags.keys())
    
    combinations=[] 
    for j in range(len(key)):
      for k in range(j+1,len(key)):
        combinations.append([key[j],key[k]])
    
    
    features=[]
    count=0
    
    for j in combinations:
      for k in paragraph:
        if((j[0]==k[0] and j[1]==k[1]) or (j[0]==k[1] and j[1]==k[0])):
          features.append([tags[j[0]],tags[j[1]],k[2],k[3],k[4]])
        elif(count<=5):
          features.append([tags[j[0]],tags[j[1]],k[2],"norelation",k[4]])
          count+=1
    y_train.append(features)
    
    
  return y_train
  
  
    
  
  
  
  


In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
tokeniz = BertTokenizer.from_pretrained('bert-base-uncased')
mdl = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

In [ ]:
val = os.listdir(TEXT_PATH)

In [ ]:
y_val=data_read(val,'dev',mdl,tokeniz)

In [ ]:
len(y_val)

28

In [ ]:
X1_val=[]
X2_val=[]
X3_val=[]
relation_val=[]
X4_val=[]

In [ ]:
for i in range(len(y_val)):
  for j in range(len(y_val[i])):
    if(len(y_val[i][j])==5):
      X1_val.append(y_val[i][j][0])
      X2_val.append(y_val[i][j][1])
      X3_val.append(y_val[i][j][2])
      relation_val.append(y_val[i][j][3]) 
      X4_val.append(y_val[i][j][4])  

In [ ]:
tok2idx = {}
tok2idx["HasProperty"] = 0
tok2idx["HasQuantity"] = 1
tok2idx["Qualifies"] = 2
tok2idx["norelation"]=3
relation_val2 = np.zeros(len(relation_val))

for i in range(len(relation_val)):
  relation_val2[i]=tok2idx[relation_val[i]]

X1_val = np.array(X1_val)
X2_val = np.array(X2_val)
X3_val = np.array(X3_val)
X4_val = np.array(X4_val)

In [ ]:
y_pred=model.predict([X1_val,X2_val,X3_val,X4_val])

In [ ]:
y_output=np.argmax(y_pred,axis=-1)

In [ ]:
count=0
total=0
p1=0
t1=0
p2=0
t2=0
for i in range(len(y_output)):
  if y_output[i]<=2 and relation_val2[i]<=2:
    if y_output[i]==relation_val2[i]:
      count+=1
    total+=1
  if y_output[i]<=2:
    if y_output[i]==relation_val2[i]:
      p1+=1
    t1+=1
  if relation_val2[i]<=2:
    if y_output[i]==relation_val2[i]:
      p2+=1
    t2+=1
prec = p1/t1
rec = p2/t2

In [ ]:
print("RELATION EXTRACTION RESULTS")
print("Accuracy " + str(count/total))
print("Precision " + str(prec))
print("Recall " + str(rec))
print("F1 Score " + str(2*prec*rec/(prec+rec)))

RELATION EXTRACTION RESULTS
Accuracy 0.9041095890410958
Precision 0.5344129554655871
Recall 0.8098159509202454
F1 Score 0.6439024390243904


In [ ]:
idx2tok = {}
idx2tok["Quantity"] = 1
idx2tok["MeasuredEntity"] = 2
idx2tok["MeasuredProperty"] = 3
idx2tok["Qualifier"] = 4

In [ ]:
tok2idx = {}
tok2idx[0] = "HasProperty"
tok2idx[1] = "HasQuantity"
tok2idx[2] = "Qualifies"
tok2idx[3] = "norelation"

In [ ]:
relation = []
for sen in lis_en:
  ans = []
  for i in range(len(sen)):
    for j in range(i+1, len(sen)):
      x1 = sen[i][1].reshape([1,768])
      x2 = sen[j][1].reshape([1,768])
      x3 = np.array([idx2tok[sen[i][0]]]).reshape([1,1])
      x4 = np.array([idx2tok[sen[j][0]]]).reshape([1,1])
      y_pred = model.predict([x1,x2,x3,x4])
      rel_pred = np.argmax(y_pred)
      ans.append([sen[i][2], sen[j][2], tok2idx[rel_pred]])
  relation.append(ans)

In [ ]:
relation